# Experiment 013: jiweiliu Kernel with CORRECT Pre-optimized Seeds

The previous lattice attempt failed because we used simple seeds (0,0,0) and (0.5,0.5,180).
This time we use the EXACT pre-optimized seeds from the jiweiliu kernel:
- Seed 1: (-4.191683864412409, -4.498489528496051, 74.54421568660419)
- Seed 2: (-4.92202045352307, -4.727639556649786, 254.5401905706735)
- a_init = 0.8744896974945239
- b_init = 0.7499641699190263

In [ ]:
import math
import numpy as np
import pandas as pd
from numba import njit
from numba.typed import List as NumbaList
import time

# Tree shape constants
TRUNK_W = 0.15
TRUNK_H = 0.2
BASE_W = 0.7
MID_W = 0.4
TOP_W = 0.25
TIP_Y = 0.8
TIER_1_Y = 0.5
TIER_2_Y = 0.25
BASE_Y = 0.0
TRUNK_BOTTOM_Y = -TRUNK_H

MAX_OVERLAP_DIST = 1.8
MAX_OVERLAP_DIST_SQ = MAX_OVERLAP_DIST * MAX_OVERLAP_DIST

print("Constants defined")

In [ ]:
@njit(cache=True)
def rotate_point(x, y, cos_a, sin_a):
    return x * cos_a - y * sin_a, x * sin_a + y * cos_a

@njit(cache=True)
def get_tree_vertices(cx, cy, angle_deg):
    angle_rad = angle_deg * math.pi / 180.0
    cos_a = math.cos(angle_rad)
    sin_a = math.sin(angle_rad)
    vertices = np.empty((15, 2), dtype=np.float64)
    pts = np.array([
        [0.0, TIP_Y], [TOP_W / 2.0, TIER_1_Y], [TOP_W / 4.0, TIER_1_Y],
        [MID_W / 2.0, TIER_2_Y], [MID_W / 4.0, TIER_2_Y], [BASE_W / 2.0, BASE_Y],
        [TRUNK_W / 2.0, BASE_Y], [TRUNK_W / 2.0, TRUNK_BOTTOM_Y],
        [-TRUNK_W / 2.0, TRUNK_BOTTOM_Y], [-TRUNK_W / 2.0, BASE_Y],
        [-BASE_W / 2.0, BASE_Y], [-MID_W / 4.0, TIER_2_Y], [-MID_W / 2.0, TIER_2_Y],
        [-TOP_W / 4.0, TIER_1_Y], [-TOP_W / 2.0, TIER_1_Y],
    ], dtype=np.float64)
    for i in range(15):
        rx, ry = rotate_point(pts[i, 0], pts[i, 1], cos_a, sin_a)
        vertices[i, 0] = rx + cx
        vertices[i, 1] = ry + cy
    return vertices

@njit(cache=True)
def polygon_bounds(vertices):
    min_x = vertices[0, 0]; min_y = vertices[0, 1]
    max_x = vertices[0, 0]; max_y = vertices[0, 1]
    for i in range(1, vertices.shape[0]):
        if vertices[i, 0] < min_x: min_x = vertices[i, 0]
        if vertices[i, 0] > max_x: max_x = vertices[i, 0]
        if vertices[i, 1] < min_y: min_y = vertices[i, 1]
        if vertices[i, 1] > max_y: max_y = vertices[i, 1]
    return min_x, min_y, max_x, max_y

@njit(cache=True)
def get_side_length(all_vertices):
    min_x = math.inf; min_y = math.inf; max_x = -math.inf; max_y = -math.inf
    for verts in all_vertices:
        x1, y1, x2, y2 = polygon_bounds(verts)
        if x1 < min_x: min_x = x1
        if y1 < min_y: min_y = y1
        if x2 > max_x: max_x = x2
        if y2 > max_y: max_y = y2
    return max(max_x - min_x, max_y - min_y)

@njit(cache=True)
def calculate_score_numba(all_vertices):
    side = get_side_length(all_vertices)
    return side * side / len(all_vertices)

print("Geometry functions defined")

In [ ]:
@njit(cache=True)
def point_in_polygon(px, py, vertices):
    n = vertices.shape[0]
    inside = False
    j = n - 1
    for i in range(n):
        xi, yi = vertices[i, 0], vertices[i, 1]
        xj, yj = vertices[j, 0], vertices[j, 1]
        if ((yi > py) != (yj > py)) and (px < (xj - xi) * (py - yi) / (yj - yi) + xi):
            inside = not inside
        j = i
    return inside

@njit(cache=True)
def segments_intersect(p1x, p1y, p2x, p2y, p3x, p3y, p4x, p4y):
    dax = p2x - p1x; day = p2y - p1y
    dbx = p4x - p3x; dby = p4y - p3y
    d1x = p1x - p3x; d1y = p1y - p3y
    d2x = p2x - p3x; d2y = p2y - p3y
    cross_b1 = dbx * d1y - dby * d1x
    cross_b2 = dbx * d2y - dby * d2x
    if cross_b1 * cross_b2 > 0: return False
    d3x = p3x - p1x; d3y = p3y - p1y
    d4x = p4x - p1x; d4y = p4y - p1y
    cross_a1 = dax * d3y - day * d3x
    cross_a2 = dax * d4y - day * d4x
    if cross_a1 * cross_a2 > 0: return False
    return True

@njit(cache=True)
def polygons_overlap(verts1, verts2, cx1, cy1, cx2, cy2):
    dx = cx2 - cx1; dy = cy2 - cy1
    if dx * dx + dy * dy > MAX_OVERLAP_DIST_SQ: return False
    min_x1, min_y1, max_x1, max_y1 = polygon_bounds(verts1)
    min_x2, min_y2, max_x2, max_y2 = polygon_bounds(verts2)
    if max_x1 < min_x2 or max_x2 < min_x1 or max_y1 < min_y2 or max_y2 < min_y1: return False
    for i in range(verts1.shape[0]):
        if point_in_polygon(verts1[i, 0], verts1[i, 1], verts2): return True
    for i in range(verts2.shape[0]):
        if point_in_polygon(verts2[i, 0], verts2[i, 1], verts1): return True
    n1 = verts1.shape[0]; n2 = verts2.shape[0]
    for i in range(n1):
        j = (i + 1) % n1
        for k in range(n2):
            m = (k + 1) % n2
            if segments_intersect(verts1[i,0], verts1[i,1], verts1[j,0], verts1[j,1],
                                  verts2[k,0], verts2[k,1], verts2[m,0], verts2[m,1]):
                return True
    return False

@njit(cache=True)
def has_any_overlap(all_vertices, centers_x, centers_y):
    n = len(all_vertices)
    for i in range(n):
        for j in range(i + 1, n):
            if polygons_overlap(all_vertices[i], all_vertices[j],
                              centers_x[i], centers_y[i], centers_x[j], centers_y[j]):
                return True
    return False

print("Overlap functions defined")

In [ ]:
@njit(cache=True)
def create_grid_vertices_extended(seed_xs, seed_ys, seed_degs, a, b, ncols, nrows, append_x, append_y):
    """Create grid with optional append rows/columns."""
    n_seeds = len(seed_xs)
    n_base = n_seeds * ncols * nrows
    n_append_x = nrows if append_x else 0
    n_append_y = ncols if append_y else 0
    n_total = n_base + n_append_x + n_append_y
    
    all_vertices = []
    centers_x = np.empty(n_total, dtype=np.float64)
    centers_y = np.empty(n_total, dtype=np.float64)
    all_degs = np.empty(n_total, dtype=np.float64)
    
    idx = 0
    for s in range(n_seeds):
        for col in range(ncols):
            for row in range(nrows):
                cx = seed_xs[s] + col * a
                cy = seed_ys[s] + row * b
                all_vertices.append(get_tree_vertices(cx, cy, seed_degs[s]))
                centers_x[idx] = cx
                centers_y[idx] = cy
                all_degs[idx] = seed_degs[s]
                idx += 1
    
    # Append in x direction (use seed 1)
    if append_x and n_seeds > 1:
        for row in range(nrows):
            cx = seed_xs[1] + ncols * a
            cy = seed_ys[1] + row * b
            all_vertices.append(get_tree_vertices(cx, cy, seed_degs[1]))
            centers_x[idx] = cx
            centers_y[idx] = cy
            all_degs[idx] = seed_degs[1]
            idx += 1
    
    # Append in y direction (use seed 1)
    if append_y and n_seeds > 1:
        for col in range(ncols):
            cx = seed_xs[1] + col * a
            cy = seed_ys[1] + nrows * b
            all_vertices.append(get_tree_vertices(cx, cy, seed_degs[1]))
            centers_x[idx] = cx
            centers_y[idx] = cy
            all_degs[idx] = seed_degs[1]
            idx += 1
    
    return all_vertices, centers_x, centers_y, all_degs

print("Grid creation defined")

In [ ]:
# PRE-OPTIMIZED SEEDS FROM JIWEILIU KERNEL (CRITICAL!)
initial_seeds = [
    (-4.191683864412409, -4.498489528496051, 74.54421568660419),
    (-4.92202045352307, -4.727639556649786, 254.5401905706735),
]
a_init = 0.8744896974945239
b_init = 0.7499641699190263

seed_xs = np.array([s[0] for s in initial_seeds], dtype=np.float64)
seed_ys = np.array([s[1] for s in initial_seeds], dtype=np.float64)
seed_degs = np.array([s[2] for s in initial_seeds], dtype=np.float64)

print("Pre-optimized seeds loaded:")
print(f"  Seed 1: ({seed_xs[0]:.6f}, {seed_ys[0]:.6f}, {seed_degs[0]:.6f})")
print(f"  Seed 2: ({seed_xs[1]:.6f}, {seed_ys[1]:.6f}, {seed_degs[1]:.6f})")
print(f"  a_init: {a_init:.6f}")
print(f"  b_init: {b_init:.6f}")

In [ ]:
# Load baseline
baseline_path = '/home/code/external_data/saspav_latest/santa-2025.csv'

def load_submission_data(filepath):
    df = pd.read_csv(filepath)
    all_xs = []; all_ys = []; all_degs = []
    for n in range(1, 201):
        prefix = f"{n:03d}_"
        group = df[df["id"].str.startswith(prefix)].sort_values("id")
        for _, row in group.iterrows():
            x = float(row["x"][1:]) if isinstance(row["x"], str) else float(row["x"])
            y = float(row["y"][1:]) if isinstance(row["y"], str) else float(row["y"])
            deg = float(row["deg"][1:]) if isinstance(row["deg"], str) else float(row["deg"])
            all_xs.append(x); all_ys.append(y); all_degs.append(deg)
    return np.array(all_xs), np.array(all_ys), np.array(all_degs)

baseline_xs, baseline_ys, baseline_degs = load_submission_data(baseline_path)

def calculate_total_score(all_xs, all_ys, all_degs):
    total = 0.0; idx = 0
    for n in range(1, 201):
        vertices = [get_tree_vertices(all_xs[idx + i], all_ys[idx + i], all_degs[idx + i]) for i in range(n)]
        total += calculate_score_numba(vertices)
        idx += n
    return total

baseline_total = calculate_total_score(baseline_xs, baseline_ys, baseline_degs)
print(f"Baseline total score: {baseline_total:.6f}")

In [ ]:
# Generate grid configurations (from jiweiliu kernel)
grid_configs = []
for ncols in range(2, 11):
    for nrows in range(ncols, 15):
        for append_x in [False, True]:
            for append_y in [False, True]:
                n_trees = 2 * ncols * nrows + (nrows if append_x else 0) + (ncols if append_y else 0)
                if 20 <= n_trees <= 200:
                    grid_configs.append((ncols, nrows, append_x, append_y, n_trees))

# Remove duplicates and sort by n_trees
grid_configs = list(set(grid_configs))
grid_configs.sort(key=lambda x: x[4])

print(f"Generated {len(grid_configs)} grid configurations")
print(f"Tree counts range: {grid_configs[0][4]} to {grid_configs[-1][4]}")

In [ ]:
# Warm up numba
print("Compiling numba functions...")
t0 = time.time()
_ = create_grid_vertices_extended(seed_xs, seed_ys, seed_degs, a_init, b_init, 2, 2, False, False)
print(f"Compilation done in {time.time() - t0:.1f}s")

In [ ]:
# Test the pre-optimized seeds with a few grid configurations
print("\nTesting pre-optimized seeds with grid configurations...")
print("="*60)

results = {}
for ncols, nrows, append_x, append_y, n_trees in grid_configs[:20]:  # Test first 20
    all_vertices, centers_x, centers_y, all_degs_grid = create_grid_vertices_extended(
        seed_xs, seed_ys, seed_degs, a_init, b_init, ncols, nrows, append_x, append_y)
    
    # Check for overlaps
    if has_any_overlap(all_vertices, centers_x, centers_y):
        continue
    
    # Calculate score
    score = calculate_score_numba(all_vertices)
    
    # Get baseline score for this N
    idx = sum(range(1, n_trees))
    baseline_vertices = [get_tree_vertices(baseline_xs[idx + i], baseline_ys[idx + i], baseline_degs[idx + i]) for i in range(n_trees)]
    baseline_score = calculate_score_numba(baseline_vertices)
    
    if score < baseline_score:
        improvement = baseline_score - score
        print(f"N={n_trees} [{ncols}x{nrows}]: lattice={score:.6f}, baseline={baseline_score:.6f}, IMPROVED by {improvement:.6f}")
        results[n_trees] = {
            'score': score,
            'baseline_score': baseline_score,
            'improvement': improvement,
            'config': (ncols, nrows, append_x, append_y),
            'centers_x': centers_x.copy(),
            'centers_y': centers_y.copy(),
            'all_degs': all_degs_grid.copy()
        }
    else:
        print(f"N={n_trees} [{ncols}x{nrows}]: lattice={score:.6f}, baseline={baseline_score:.6f} (no improvement)")

In [ ]:
# Summary
print("\n" + "="*60)
print("JIWEILIU CORRECT SEEDS RESULTS")
print("="*60)

if results:
    total_improvement = sum(r['improvement'] for r in results.values())
    print(f"\nImproved {len(results)} configurations:")
    for n, r in sorted(results.items()):
        print(f"  N={n}: {r['baseline_score']:.6f} -> {r['score']:.6f} (improvement: {r['improvement']:.6f})")
    print(f"\nTotal improvement: {total_improvement:.6f}")
else:
    print("\nNo improvements found.")
    print("The pre-optimized seeds still don't beat the baseline.")

In [ ]:
# Copy baseline to submission (since likely no improvements)
import shutil
shutil.copy('/home/code/external_data/saspav_latest/santa-2025.csv', '/home/submission/submission.csv')
print("Copied baseline to submission")